# Emerging Technologies QA with pdf data

* Date: Dec 03, 2023
* Author: Dawan Savage Bell
* W0465310
* Mileston 10



### Resources used 
https://medium.com/@alinaeem_57283/building-a-pdf-querying-bot-with-langchain-edb71ebedcc

In [4]:
import os
import json
from langchain import HuggingFaceHub

print("imports completed")

try:
    # read in json file and store hugging face key
    with open("./keys/keys.json") as f:
        hug_token = json.load(f)["huggingface_token"]
        os.environ["HUGGINGFACEHUB_API_TOKEN"] = hug_token
        print("environment set")
except Exception as e:
    print(e)
    print("token not set")

imports completed
environment set


# Creating LLM based off of bloom model from Huggingface

* from BigScience Workshop. 
* architecture of Bloom similar to GPT3, trained on 46 different languages and 13 programming languages.
* https://huggingface.co/docs/transformers/model_doc/bloom

In [5]:
# get the BLOOm Model from HuggingFace
llm = HuggingFaceHub(repo_id="bigscience/bloom",
                     model_kwargs={"temperature":0.1,
                                   "max_length":100})

c:\Users\Savag\.conda\envs\nscc_env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


# Text Splitter
* simple splitting the text by each character.
* https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/character_text_splitter

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

print("imports complete")

imports complete


In [6]:
# loading in document
pdf_path = "./docs/albert_einstein_relativity.pdf"
loader = PyPDFLoader(pdf_path)
doc = loader.load()

# split the doc into 
# chunk size is the number of characters
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(doc)

NameError: name 'PyPDFLoader' is not defined

# Embeddings
* translating the chunk of texts to series of numbers (embeddings)
* using a pre-trained model for generating the embeddings
* resource:["https://pub.aimind.so/llm-embeddings-explained-simply-f7536d3d0e4b"]

In [23]:
# selecting the embedding 
from langchain.embeddings import HuggingFaceEmbeddings
print("imports complete")

imports complete


In [24]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

# Vector DB
* https://python.langchain.com/docs/modules/data_connection/vectorstores/
* resource:["https://www.pinecone.io/learn/vector-database/"]

In [30]:
from langchain.vectorstores import Chroma

In [31]:
# create the vectorestore to use index
db = Chroma.from_documents(texts, embeddings)

In [64]:
# expose the index in a retriever obj/instance
from langchain.chains import RetrievalQA

try:
    retriever = db.as_retriever(search_type="similarity",
                                search_kwargs={"k": 3} 
                                )
except Exception as e:
    print("error: ", e)
    
    
try:
    # Initialize RetrievalQA with the required parameters
    # Note: return_scores is removed as it's not a valid parameter
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
except Exception as e:
    # Print any error that occurs during the initialization
    print("error: ", e)

In [69]:
try:
    # ask a question
    question = "what is relativity?"
    answers = qa({"query":question})

    # print the type and value of answers
    # print(type(answers))
    # print(answers.keys())
    print(answers["query"])
    
    print(answers["result"])    
    
    #print(answers)
except Exception as e:
    # Print any error that occurs during the execution
    print("An error occurred: ", e)

An error occurred:  (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 38563f94-80e1-4725-bb77-d4482160f515)')
